# TensorFlow Tutorial - 4. MLP

본 문서는 TensorFlow 를 사용하여 Deep Learning을 구현하기 위한 기초적인 실습 자료이다. 첫 번째 파트에서는 tensorflow에 대한 기본적인 설명과 deep learning 예제를 다루어보고, 두 번째 파트에서는 오픈소스를 활용한 Deep Reinforcement Learning 을 실습해보는 시간을 갖는다. 마지막으로는 TensorFlow로 구현되고 공개된 여러 오픈소스를 둘러본다.

The code and comments are written by Dong-Hyun Kwak <imcomking@gmail.com>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.




# Agenda
## [1. Tensorflow Intro](../TensorflowIntro/TensorflowIntro.ipynb)
## [2. Basic](../Basic/Basic.ipynb)
## [3. Gradient Descent](../GradientDescent/GradientDescent.ipynb)
## [4. MLP](../MLP/MLP.ipynb)
## [5. RNN](../RNN/RNN.ipynb)
## [6. CNN](../CNN/CNN.ipynb)
## [7. Reinforcemet Learning](../ReinforcemetLearning/ReinforcemetLearning.ipynb)


#  4. MLP
<img src = "tensorboard_mlp.png">
우선 기본적인 tf의 문법과 형식을 익히기 위해, 가장 기본적인 Deep Learning인 MLP 예제 코드를 실습해보자.

### Multi-Layer Perceptron
Multi-Layer Perceptron, 이하 MLP는 다음과 같은 구조를 가진 모델이다. Convolutional Neural Networks와 달리 굉장히 layer간의 연결이 빽빽하게 가득 차 있어, dense layer 혹은 fully connected layer라는 이름으로도 불리고 있다.


<img src = "mlp.png">
(출처: http://blog.refu.co/?p=931)
tf를 이용해서 MNIST 데이터를 MLP로 분류하는 코드를 작성해보자.
가장 먼저 사용할 라이브러리를 import 해준다.

In [1]:
%matplotlib inline
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
print (tf.__version__)

1.1.0


그리고 실습에서 사용할 MNIST 데이터를 다음과 같이 다운로드 받는다.


In [2]:
# download the mnist data.
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [19]:
import numpy as np
for k in range(10):
    print (mnist.train.labels[k])
    img = mnist.train.images[k]
    for i in range(28):
        l = ""
        for j in range(28):
            if img[i*28+j] > 0.5:
                l = l+"O"
            else:
                l = l+" "
        print(l)

[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
                            
                            
                            
                            
                            
                            
                            
                            
    OOOOOOOOOOOOOO          
  OOOOOOOOOOOOOOOOO         
  OOOOOOOO     OOOOO        
                 OOO        
                  OOO       
                  OOO       
                 OOOO       
             OOOOOOOOO      
           OOOOOOOOOOO      
        OOOOOOOO  OOO       
       OOOO       OOO       
      OOO         OOO       
                  OOO       
                 OOO        
                OOOO        
                OOO         
                OO          
               OOO          
                            
                            
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
                            
                            
                            
                 

이제 mnist 데이터를 저장시킬 x와 y_target 변수를 선언해야한다.
tf.placeholder는 우리가 원하는 데이터를 Computation Graph에 입력시켜주는 역할을 하는 변수이다. 즉 input 을 받기 위한 변수라고 생각할 수 있다.

In [4]:
# placeholder is used for feeding data.
x = tf.placeholder("float", shape=[None, 784]) # none represents variable length of dimension. 784 is the dimension of MNIST data.
y_target = tf.placeholder("float", shape=[None, 10]) # shape argument is optional, but this is useful to debug.

데이터를 넣을 변수를 생성했으니, 이제 실제 Neural Network에서 사용할 변수들을 생성하고, 이들을 이용해 Computation Graph를 그려본다.

In [5]:
# all the variables are allocated in GPU memory
W1 = tf.Variable(tf.zeros([784, 256]))      # create (784 * 256) matrix
b1 = tf.Variable(tf.zeros([256]))           # create (1 * 256) vector
weighted_summation1 = tf.matmul(x, W1) + b1 # compute --> weighted summation
h1 = tf.sigmoid( weighted_summation1 )      # compute --> sigmoid(weighted summation)

# Repeat again
W2 = tf.Variable(tf.zeros([256, 10]))        # create (256 * 10) matrix
b2 = tf.Variable(tf.zeros([10]))             # create (1 * 10) vector
weighted_summation2 = tf.matmul(h1, W2) + b2 # compute --> weighted summation
y = tf.nn.softmax(weighted_summation2)       # compute classification --> softmax(weighted summation)

위의 과정까지 완료된 경우, 변수 y는 3층짜리 Neural Network(MLP)의 연산 결과가 저장되도록 되어있다. 이제 이 MLP를 학습하고 실행시켜보자.

먼저 tf에는 Session이라는 개념이 있다. 이는 간단히 말해 하나의 Computation Graph가 실행되기 위한 계산 단위를 의미한다. 즉 하나의 session에는 보통 하나의 graph가 담겨있다. 이를 생성해보자.

In [6]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True))) # open a session which is a envrionment of computation graph.
sess.run(tf.initialize_all_variables())# initialize the variables

Instructions for updating:
Use `tf.global_variables_initializer` instead.


위의 tf.initialize_all_variables() 는 앞서 생성했던 변수들을 사용하기 위해서 반드시 실행해야하는 초기화 단계이다. 이 연산은 session에 의해서 실행되므로, 이와 같이 sess.run() 함수를 이용해 실행시켜준다.

이제 MLP를 학습시키기 위한 미분 계산과 같은 수학적인 연산들을 정의해보자.

In [7]:
# define the Loss function
cross_entropy = -tf.reduce_sum(y_target*tf.log(y))

cross entorpy는 MLP의 분류 모델에서 사용하는 에러함수이다. 간단히 말해 이 cross entropy는 MLP가 예측한 y 값이 실제 데이터와 다른 정도를 확률적으로 측정한다고 생각해볼 수 있다.

MLP가 학습 되기위해서는 이 에러함수가 최대한 작은 값을 내도록 만들어야한다. 그래서 이 에러함수를 각각의 변수들로 편미분하여 gradient를 계산하고, 에러가 최소가 되는 변수값을 찾아가는 것이 바로 MLP의 학습 알고리즘이다.

이를 구현하려면 원래 미분 후 에러가 줄어드는 방향으로 변수를 이동시키는 과정등을 직접 코딩해야하지만, 자동 미분을 제공하는 tf에서는 단 한줄로 구현할 수 있다.



In [8]:
# define optimization algorithm
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

이제 학습 알고리즘의 구현이 끝났고, MLP가 잘 학습하고 있는지 성능을 측정하기 위한 정확도 계산을 정의해보자.

In [9]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
# correct_prediction is list of boolean which is the result of comparing(model prediction , data)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
# tf.cast() : changes true -> 1 / false -> 0
# tf.reduce_mean() : calculate the mean

correct_prediction 는 뭔가 수식이 복잡해 보이지만 사실 매우 간단한 계산이다. tf.argmax()는 y 벡터 중에서 가장 값이 큰 index를 알려주는 함수이다. 즉 모델이 예측한 class, y와 실제 데이터에 labeling된 y_target을 비교하여 같으면 true, 다르면 false를 내도록 계산한 것이다.

그리고 accuracy는 앞서 계산한 true/false 리스트를 1과 0으로 변환한 뒤, 이를 평균낸 것이다.

자 그러면 이제 필요한 모든 Computation Graph를 정의하였다. 이제 이를 session을 이용하여 실행만 시키면 된다.


우선은 Ctrl + Enter를 눌러 다음 코드를 실행 시켜보자.

In [11]:
# training the MLP
for i in range(5001): # minibatch iteraction
    batch = mnist.train.next_batch(100) # minibatch size
    sess.run(train_step, feed_dict={x: batch[0], y_target: batch[1]}) # feed data into placeholder x, y_target

    if i%500 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1]})
        print ("step %d, training accuracy: %.3f"%(i, train_accuracy))

# for given x, y_target data set
print  ("test accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.test.images, y_target: mnist.test.labels}))
sess.close()
tf.reset_default_graph()

step 0, training accuracy: 0.140
step 500, training accuracy: 0.390
step 1000, training accuracy: 0.530
step 1500, training accuracy: 0.760
step 2000, training accuracy: 0.890
step 2500, training accuracy: 0.900
step 3000, training accuracy: 0.870
step 3500, training accuracy: 0.930
step 4000, training accuracy: 0.930
step 4500, training accuracy: 0.940
step 5000, training accuracy: 0.910
test accuracy: 0.905


총 5000번의 minibatch iteraction을 실행하고, 500번 마다 학습 정확도를 측정, 그리고 마지막에는 테스트 정확도를 측정하고 있다.

코드는 간단한 구조이다. for 문이 전체 iteration을 실행하고, 맨 처음 가져왔던 mnist 데이터를 100개씩 가져와서 place_holder에 넣어준다. 그리고 sess.run()을 통해 위에서 정의했던 학습 알고리즘과 정확도 계산을 실행시킨다.

이제 우리가 만든 MLP의 구조를 직접 눈으로 확인해보자.

이제 우리가 만든 MLP의 구조를 직접 눈으로 확인해보자.

### TensorBoard 설정하기
TensorFlow는 TensorBoard라는 매우 강력한 visualization tool을 제공한다. 이를 사용하면 웹브라우저 형태로 사용자가 모델의 구조를 눈으로 확인하고, 파라미터 값의 변화를 살펴보는 등의 직관적인 분석이 가능하다.

이를 활용해 방금 만들었던 MLP를 분석해보자. 그러려면 다음의 사항을 반영해 코드를 수정하여야 한다.

* **변수들의 이름 지어주기**

* **변수들의 Summary 생성**

* **변수들의 Summary 기록**

아래의 코드는 위의 3가지 사항을 모두 반영하고, MLP 코드를 하나의 함수로 정리한 코드이다. 세세한 차이는 위에서 우리가 짰던 코드와 비교를 하면 파악이 가능하다.

In [1]:
%matplotlib inline
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()

def MLP():
    # download the mnist data.
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True) 


    # placeholder is used for feeding data.
    x = tf.placeholder("float", shape=[None, 784], name = 'x') # none represents variable length of dimension. 784 is the dimension of MNIST data.
    y_target = tf.placeholder("float", shape=[None, 10], name = 'y_target') # shape argument is optional, but this is useful to debug.


    # all the variables are allocated in GPU memory
    W1 = tf.Variable(tf.zeros([784, 256]), name = 'W1')   # create (784 * 256) matrix
    b1 = tf.Variable(tf.zeros([256]), name = 'b1')        # create (1 * 256) vector
    h1 = tf.sigmoid(tf.matmul(x, W1) + b1, name = 'h1')   # compute --> sigmoid(weighted summation)

    # Repeat again
    W2 = tf.Variable(tf.zeros([256, 10]), name = 'W2')     # create (256 * 10) matrix
    b2 = tf.Variable(tf.zeros([10]), name = 'b2')          # create (1 * 10) vector
    y = tf.nn.softmax(tf.matmul(h1, W2) + b2, name = 'y')  # compute classification --> softmax(weighted summation)


    # define the Loss function
    cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')


    # define optimization algorithm
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)



    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
    # correct_prediction is list of boolean which is the result of comparing(model prediction , data)


    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
    # tf.cast() : changes true -> 1 / false -> 0
    # tf.reduce_mean() : calculate the mean



    # create summary of parameters
    tf.summary.histogram('weights_1', W1)
    tf.summary.histogram('weights_2', W2)
    tf.summary.histogram('y', y)
    tf.summary.scalar('cross_entropy', cross_entropy)
    merged = tf.summary.merge_all()

    # Create Session
    sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) # open a session which is a envrionment of computation graph.
    sess.run(tf.global_variables_initializer())# initialize the variables
    
    summary_writer = tf.summary.FileWriter("/tmp/mlp",sess.graph)

    # training the MLP
    for i in range(5001): # minibatch iteraction
        batch = mnist.train.next_batch(100) # minibatch size
        sess.run(train_step, feed_dict={x: batch[0], y_target: batch[1]}) # placeholder's none length is replaced by i:i+100 indexes

        if i%500 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1]})
            print ("step %d, training accuracy: %.3f"%(i, train_accuracy))



            # calculate the summary and write.
            summary = sess.run(merged, feed_dict={x:batch[0], y_target: batch[1]})
            summary_writer.add_summary(summary , i)

    # for given x, y_target data set
    print  ("test accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.test.images, y_target: mnist.test.labels}))
    sess.close()

MLP()

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
step 0, training accuracy: 0.140
step 500, training accuracy: 0.360
step 1000, training accuracy: 0.340
step 1500, training accuracy: 0.430
step 2000, training accuracy: 0.330
step 2500, training accuracy: 0.370
step 3000, training accuracy: 0.310
step 3500, training accuracy: 0.350
step 4000, training accuracy: 0.380
step 4500, training accuracy: 0.350
step 5000, training accuracy: 0.380
test accuracy: 0.3819


### TensorBoard 실행하기
위와 같이 코드를 수정했다면, 이제 리눅스 shell로 이동한 후, tensorboard를 실행시킨다.
혹은 IPython에서 new -> terminal을 클릭하여 아래의 명령을 실행할 수도 있다.


tensorboard --logdir=/tmp/mlp --port=6006


(만약 위 명령어 실행시 문제가 생기는 경우 다음을 실행)
<br>cd tensorflow/tensorflow/tensorboard
<br>python tensorboard.py --logdir=/tmp/mlp --port=6006


그다음 [IP]:6006/#graphs 에 접속하면 아래와 같은 그림을 볼 수 있다.
<br>(ex) http://192.168.99.100:6006/#graphs

<img src = "mlp_total.png">
